In [1]:
import sys
import os
import numpy as np
import pandas as pd
import pickle
import math
from astropy.io import fits
from sedpy.observate import load_filters
from prospect import prospect_args
from prospect.fitting import fit_model
from prospect.models.sedmodel import SpecModel, LineSpecModel
from prospect.io import write_results as writer
from astropy.cosmology import Planck15 as cosmo
from astropy.table import Table
from prospect.likelihood import NoiseModel
from prospect.likelihood.kernels import Uncorrelated
from prospect.sources.galaxy_basis import CSPSpecBasis
import matplotlib.pyplot as plt
from prospect.utils.obsutils import fix_obs
import prospect.io.read_results as reader

In [2]:
path_wdir = "/Users/amanda/Desktop/Paper/technical/"
path_data = os.path.join(path_wdir, "data/")
path_output = os.path.join(path_wdir, "prospector/")
path_mock = '/Users/amanda/Desktop/Paper/technical/data/mock/'


In [3]:
file = [0,1,2,3,4,8]

In [4]:
filename = 'test2.pickle'

# open the file in write mode and save the dictionary as a pickle
with open(path_output + filename, 'wb') as f:
    pickle.dump(file, f)


In [5]:

# open the file in read mode and load the pickled dictionary
with open(path_output + filename, 'rb') as f:
    output = pickle.load(f)

In [6]:
output

[0, 1, 2, 3, 4, 8]

In [7]:
from astropy.io import fits
import pandas as pd
import numpy as np

# Read the CSV file using pandas
# It is important that the data itself does not contain any data type that is not convertable to float.
# If it does, a dictionary is required to translate the data.
df = pd.read_csv(path_data+'galaxies_nostring.csv')
df = df.astype(float)

# Convert the dataframe to a numpy structured array
data = df.to_records(index=False)

# Create a new FITS binary table HDU
hdu = fits.BinTableHDU.from_columns(data)

# Write the FITS file
hdu.writeto(path_data+'galaxies_nostring.fits', overwrite=True)

In [8]:

num_to_id_translate = {'3307':'J003601+003307',
                '15440':'J004743+015440',
                '223':'J011309+000223',
                '52044':'J012217+052044',
                '145935':'J012910+145935',
                '414608':'J072326+414608',
                '472607':'J080425+472607',
                '414146':'J081112+414146',
                '211459':'J081409+211459',
                '182052':'J082652+182052',
                '480541':'J083440+480541',
                '392925':'J090918+392925',
                '183108':'J091113+183108',
                '523960':'J091207+523960',
                '505009':'J091208+505009',
                '315221':'J091703+315221',
                '395714':'J092552+395714',
                '510925':'J093355+510925',
                '593244':'J094001+593244',
                '405249':'J095236+405249',
                '235709':'J095700+235709',
                '202508':'J095838+202508',
                '523251':'J101401+523251',
                '633308':'J102615+633308',
                '635317':'J103344+635317',
                '452718':'J103816+452718',
                '474357':'J105117+474357',
                '523753':'J105331+523753',
                '485724':'J105525+485724',
                '475204':'J110452+475204',
                '412052':'J112224+412052',
                '524509':'J112848+524509',
                '493525':'J112933+493525',
                '651341':'J113304+651341',
                '312559':'J115855+312559',
                '382422':'J115959+382422',
                '305326':'J120934+305326',
                '453930':'J121915+453930',
                '63556':'J123519+063556',
                '212716':'J124033+212716',
                '21540':'J124423+021540',
                '444902':'J124619+444902',
                '123403':'J124835+123403',
                '464535':'J124911+464535',
                '255609':'J125503+255609',
                '410221':'J125718+410221',
                '451057':'J130059+451057',
                '510451':'J130128+510451',
                '422638':'J130559+422638',
                '214817':'J131037+214817',
                '104739':'J131419+104739',
                '510309':'J131904+510309',
                '421824':'J132633+421824',
                '573315':'J132937+573315',
                '112848':'J134559+112848',
                '662438':'J135008+662438',
                '612115':'J140333+612115',
                '434435':'J141013+434435',
                '461937':'J144010+461937',
                '370512':'J151707+370512',
                '572442':'J154050+572442',
                '403325':'J155945+403325',
                '81959':'J160437+081959',
                '313054':'J164607+313054',
                '495751':'J164849+495751',
                '542133':'J172010+542133'}
id_to_num_translate = {v: k for k, v in num_to_id_translate.items()}


In [9]:
datas = fits.open(os.path.join(path_data, 'galaxies_nostring.fits'))
data = datas[1].data

# How to distort a data point

In [10]:
fesc = 0.013
sigma = 0.003
noise = np.random.normal(0, sigma)
distorted_fesc = fesc + noise

In [11]:
draws = np.zeros(100)

# Generate random draws
for i in range(100):
    draws[i] = np.random.normal(0, sigma)

# Parameters - Thetas

In [12]:
parnames = ['logzsol', 'dust2', 'logmass', 'logsfr_ratios_1', 'logsfr_ratios_2', 'logsfr_ratios_3', 'logsfr_ratios_4', 'logsfr_ratios_5', 'dust_index', 'dust1_fraction']

In [13]:

def build_sps(zcontinuous=1, sfh_template="continuity_sfh", compute_vega_mags=False, **extras):
    if (sfh_template == "continuity_sfh") or (sfh_template == "dirichlet_sfh"):
        from prospect.sources import FastStepBasis
        sps = FastStepBasis(zcontinuous=zcontinuous,
                            compute_vega_mags=compute_vega_mags,
                            reserved_params=['tage', 'sigma_smooth'])
    else:
        from prospect.sources import CSPSpecBasis
        sps = CSPSpecBasis(zcontinuous=zcontinuous,
                           compute_vega_mags=compute_vega_mags,
                           reserved_params=['sigma_smooth'])
    return sps

In [14]:
observed = pd.read_csv('/Users/amanda/Desktop/Paper/technical/data/flury/lzlcs_optlines_obs.csv')

In [15]:
def filter_strings(strings):
    result = [string for string in strings if not string.endswith("e")el]
    return result
el_obs = filter_strings(observed.columns)

SyntaxError: invalid syntax (1283304194.py, line 2)

In [16]:
el_obs

NameError: name 'el_obs' is not defined

In [17]:
translate_el={'O2_3726A':'[OII]3726',
 'O2_3729A':'[OII]3729',
 'Ne3_3869A':'[NeIII]3870',
 'H1r_3889A':'HeI 3889',
 'Ne3_3967A':'[NeIII]3968',
 'H1r_3970A':'H 3970',
 'H1r_4102A':'H delta 4102',
 'H1r_4341A':'H gamma 4340',
 'O3_4363A':'[OIII]4364',
 'He2r_4686A':None,
 'H1r_4861A':'H beta 4861',
 'O3_4959A':'[OIII]4960',
 'O3_5007A':'[OIII]5007',
 'He1r_5876A':'HeI 5877',
 'O1_6300A':'[OI]6302',
 'N2_6548A':'[NII]6549',
 'H1r_6563A':'H alpha 6563',
 'N2_6584A':'[NII]6585',
 'S2_6716A':'[SII]6717',
 'S2_6731A':'[SII]6732'}

In [18]:
translate_name[num_to_id_translate['15440']]

NameError: name 'translate_name' is not defined

In [19]:
path_wdir = '/Users/amanda/Desktop/Paper/technical/'
path_data = os.path.join(path_wdir, 'data/')
path_output = os.path.join(path_wdir, 'prospector/')
path_flury = os.path.join(path_wdir, 'data/flury/')


In [20]:
def enumerate_strings(strings):
    enumerated_dict = {string: str(index + 1) for index, string in enumerate(strings)}
    return enumerated_dict

translate_name = enumerate_strings(id_to_num_translate.keys())
translate_name['J003601+003307']

'1'

In [21]:
phot_table='GP_Aperture_Matched_Photometry_v0.fits'
el_table="lzlcs_optlines_obs.csv"
objid=1

In [22]:
phot_cat = fits.open(os.path.join(path_flury, phot_table))
phot = phot_cat[1].data
phot[1][0]

2

In [23]:
el = Table.read(os.path.join(path_flury, el_table), format="ascii")
el.add_column([i for i in range(1,67)], name='id', index=0)
el

id,NAME,O2_3726A,O2_3726Ae,O2_3729A,O2_3729Ae,Ne3_3869A,Ne3_3869Ae,H1r_3889A,H1r_3889Ae,Ne3_3967A,Ne3_3967Ae,H1r_3970A,H1r_3970Ae,H1r_4102A,H1r_4102Ae,H1r_4341A,H1r_4341Ae,O3_4363A,O3_4363Ae,He2r_4686A,He2r_4686Ae,H1r_4861A,H1r_4861Ae,O3_4959A,O3_4959Ae,O3_5007A,O3_5007Ae,He1r_5876A,He1r_5876Ae,O1_6300A,O1_6300Ae,N2_6548A,N2_6548Ae,H1r_6563A,H1r_6563Ae,N2_6584A,N2_6584Ae,S2_6716A,S2_6716Ae,S2_6731A,S2_6731Ae
int64,str14,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1,J003601+003307,30.848469,1.6267882,39.194013,1.85677,61.752844,1.5614241,23.902108,1.251398,21.293996,1.8511538,17.924301,1.5520626,32.573245,1.4378267,61.689684,1.6840835,16.382929,1.4234521,0.0,-99.0,136.51584,2.8749645,270.69147,4.0618624,851.03992,70.19677,19.74732,2.3235349,0.0,-99.0,17.411655,9.7469951,454.49678,6.4211528,18.705467,2.8267641,0.0,-99.0,4.8370173,2.9291843
2,J004743+015440,99.95118,2.5786292,117.33518,2.5714145,65.894175,1.4575998,24.7092,1.2368046,23.484576,1.3300223,14.041572,1.0934409,32.713175,1.8441317,69.646969,2.4282756,13.513025,1.7489098,3.2612795,1.6085462,164.99566,3.1819297,306.23182,3.0985201,912.5708236,19.964407,19.101125,1.6748919,8.1248294,4.5328226,0.0,-99.0,581.37334,9.2648435,31.07848,4.2073958,30.805263,2.6246507,22.716762,2.6216319
3,J011309+000223,129.81044,7.4570595,104.4955,5.7289957,48.645583,2.6631381,16.344518,1.9476191,14.293992,2.6974809,12.367662,2.8363087,20.175669,2.2894186,44.132245,2.5256307,10.392885,6.0269999,0.0,-99.0,86.052626,2.9408101,137.41586,2.5002981,414.18911,6.7059248,9.1646792,2.0910171,3.4130506,2.375434,7.3151559,4.820293,236.63029,4.5145126,9.9410234,2.2024829,40.954879,8.1482302,21.400612,3.6748428
4,J012217+052044,45.142712,2.4061713,56.985604,2.3066852,49.116555,2.1975888,18.816708,1.763831,15.582407,1.5577252,13.072769,2.0371334,24.998753,1.8565243,48.28456,2.6301346,13.889922,2.5520472,0.0,-99.0,106.11932,2.6259933,207.50764,4.1958245,617.4594,7.7631898,16.972526,3.6921942,0.0,-99.0,7.4445179,6.4020644,310.42633,13.854743,12.56388,5.065243,6.1359202,4.0039381,0.0,-99.0
5,J012910+145935,276.60776,10.311352,168.55332,6.3715213,59.567164,3.538896,32.730851,3.059031,22.760236,3.7187031,25.093901,4.0876691,44.750847,2.1164488,91.683683,3.540858,7.7279691,4.7998386,0.0,-99.0,230.23128,3.0637697,284.89957,3.4602202,862.30319,8.9109926,33.081378,3.3773743,18.26653,3.4204616,25.170278,4.9831799,760.02105,11.621608,70.599166,6.955919,59.882423,4.4873086,37.757842,4.8497625
6,J072326+414608,123.36735,4.4800503,113.5584,3.8257978,57.239538,1.6243399,24.506436,1.1926199,18.854429,1.4160764,17.95819,1.1620583,34.885696,1.5326853,71.347201,1.6356167,9.4803453,1.0230134,3.5597841,2.1755986,167.61405,3.6435579,277.14875,3.9127262,825.903275,12.035654,21.782554,1.7098247,10.809531,1.2669232,10.447496,1.5716401,508.56689,13.800866,29.903918,1.6411269,36.23451,1.2536137,27.059068,1.2331236
7,J080425+472607,4.0327091,0.80419429,3.7627858,0.88413801,15.887941,0.94976773,8.2493884,0.77386584,4.6168168,0.95428186,7.8282326,0.85276416,11.140076,1.1697717,21.753691,1.9154514,6.8734645,1.3004344,0.0,-99.0,51.222154,1.3879705,80.253615,1.7538652,239.1557727,3.4947038,5.2085186,3.133307,0.0,-99.0,0.0,-99.0,158.64276,4.3740875,0.0,-99.0,0.0,-99.0,0.0,-99.0
8,J081112+414146,17.094261,1.1439201,23.953992,1.227835,25.288982,1.5853496,9.7787002,0.997191,9.3656289,1.138548,7.531339,0.86360814,13.219281,1.0063095,28.087701,1.2847884,8.4413669,1.194478,2.6769266,0.96335368,57.738654,1.3468505,132.11593,2.085111,393.7054714,3.6040003,7.4347146,0.94963887,0.68915779,2.4965073,0.0,-99.0,158.407,3.2472419,0.0,-99.0,8.0552802,2.3407854,0.0,-99.0
9,J081409+211459,339.99141,6.620331,415.00061,6.0214874,94.932226,3.2180826,39.662709,2.8148596,20.95812,3.1162267,22

In [24]:
translate_name_rev = {v: k for k, v in translate_name.items()}
filternames = ['sdss_u0','sdss_g0','sdss_r0','sdss_i0','sdss_z0','galex_FUV','galex_NUV']


In [25]:
phot['p']

KeyError: "Key 'p' does not exist."

In [26]:
EL_info = pd.read_csv('/Users/amanda/opt/anaconda3/envs/astro/lib/python3.10/site-packages/fsps/data/emlines_info.dat', header=None, sep = ',')


In [27]:
def build_obs(objid=1, EL_info=EL_info, el_table="lzlcs_optlines_obs.csv", phot_table='GP_Aperture_Matched_Photometry_v0.fits', err_floor=0.05, err_floor_el=0.05, filternames = filternames, **kwargs):
    
    # choose galaxy and read in the data
    phot_cat = fits.open(os.path.join(path_flury, phot_table))
    phot = phot_cat[1].data

    el = Table.read(os.path.join(path_flury, el_table), format="ascii")
    el.add_column([i for i in range(1,67)], name='id', index=0) #uniform reference for name now possible

    #two different inputs are possible for the objid, so we have to combine them into one ID we can work with:
    if objid>70:
        idx_gal = translate_name[num_to_id_translate[str(objid)]]
    elif objid==0:
        print('ERROR = objid can not be zero, the enumerations starts at 1')
    else:
        idx_gal = int(phot[objid-1][0])

    #id will be used to access the photometry, here everything starts from 0

    id = idx_gal-1

    #control line gets printed:
    print('GALAXIE: '+translate_name_rev[str(idx_gal)]+', ID: ' + str(idx_gal) + ', PHOTOMETRY: ' + phot_table + ', EMISSION_LINES: '+ el_table)

    #create all the lists I need for the obs dictionary:

#-------------------------------PHOTOMETRY-------------------------------------
    maggies = []
    maggies_unc = []
    phot_mask = []
    filters = []

    #MAGGIES AND MAGGIES_UNC:

    fil = ['FUV', 'NUV', 'U', 'G', 'R', 'I', 'Z']
    for x in fil:
        if phot['probPSF'][0]==1:
            if phot['psf_mag_'+x][id] > 0:
                m = 10**((phot['psf_mag_'+x][id]-8.9)/(-2.5))
                m_err = np.abs(m - 10**(((phot['psf_mag_'+x][id]+phot['psf_magerr_'+x][id])-8.9)/(-2.5)))
                maggies.append(m/3631)
                maggies_unc.append(m_err/3631)
            else:
                maggies.append(None)
                maggies_unc.append(None)
        else:
            if phot['aper_mag_3p1_'+x][id] > 0:
                m = 10**((phot['aper_mag_3p1_'+x][id]-8.9)/(-2.5))
                m_err = np.abs(m - 10**(((phot['aper_mag_3p1_'+x][id]+phot['aper_magerr_3p1_'+x][id])-8.9)/(-2.5)))
                maggies.append(m/3631)
                maggies_unc.append(m_err/3631)
            else:
                maggies.append(None)
                maggies_unc.append(None)
        
    maggies = np.array(maggies)
    maggies_unc = np.array(maggies_unc)


    #PHOT_MASK:

    for i in range(len(maggies)):
        if maggies[i] == None:
            phot_mask.append(False)
        else:
            phot_mask.append(True)
    
    phot_mask = np.array(phot_mask)


    #FILTERS:

    filternames = np.array(filternames)

#-------------------------------EMISSION LINES-------------------------------------
    
    wavelength = []
    spectrum = []
    unc = []
    mask = []


    #SPECTRUM, UNC AND WAVELENGTH:

    for i_col in el.columns:
        if ('Ae' not in i_col) & ('NAME' not in i_col) & ('id' not in i_col):
            spectrum.append(el[i_col][id])
            unc.append(el[i_col+'e'][id])
            wavelength.append(translate_el[i_col])

    spectrum = np.array(spectrum)
    unc = np.array(unc)
    wavelength = np.array(wavelength)
    #unc = [spectrum[i]/10 for i in range(len(spectrum))]

    #MASK:

    for i in range(len(wavelength)):
        if wavelength[i]==None:
            mask.append(False)
        else:
            mask.append(True)

    mask = np.array(mask)

    #put the names of the lines in a list, create list with the idices of the positions our EL have in the emission_info.dat file

    line_info = np.genfromtxt(os.path.join(os.getenv("SPS_HOME"), "data/emlines_info.dat"), dtype=[('wave', 'f8'), ('name', '<U20')], delimiter=',')
    linelist = line_info["name"].tolist()
    line_indices = []
    for n in wavelength:
        if n==None:
            continue
        else:
            line_indices.append(linelist.index(n))


    #----------------------------------NOW CREATE OBS DICTIONARY----------------------------------#

    # set up obs dictionary

    obs = {}

    # This is a list of sedpy filter objects.
    # See the sedpy.observate.load_filters command for more details on its syntax.

    obs['filters'] = load_filters(filternames)
    obs['wave_effective'] = [f.wave_effective for f in obs['filters']]

    filter_width_eff = [f.effective_width for f in obs['filters']]

    # This is a list of maggies, converted from mags.
    # It should have the same order as `filters` above.
    obs['maggies'] = maggies

    # You should use real flux uncertainties (incl. error floor)
    mags_err_final = np.clip(maggies_unc, np.abs(maggies) * err_floor, np.inf)
    obs['maggies_unc'] = mags_err_final

    # Here we mask out any NaNs or infs
    # mask bands below Lyman break (if redshift fixed)
    #obs['phot_mask'] = np.isfinite(np.squeeze(maggies)) & (mags_err < 1e4) & (mags != 1.0)
    obs['phot_mask'] = phot_mask

    # We do not have a spectrum, so we set some required elements of the obs dictionary to None.
    # (this would be a vector of vacuum wavelengths in angstroms)
    obs["wavelength"] = line_info["wave"][np.array(line_indices)]

  

    # (this would be the spectrum in units of maggies)
    obs["spectrum"] = spectrum
    obs["line_ind"] = line_indices

    # (spectral uncertainties are given here)
    #el_unc_final = np.clip(el_unc, np.abs(el_flux) * err_floor_el, np.inf)
    #obs['unc'] = el_unc_final
    obs['unc'] = unc
    # (again, to ignore a particular wavelength set the value of the
    #  corresponding elemnt of the mask to *False*)
    obs['mask'] = mask
    #obs['mask'] = [True for i in range(len(obs['spectrum']))]
    # Add unessential bonus info.  This will be stored in output
    obs['cat_row'] = id
    obs['id'] = idx_gal
    obs['z_spec'] = phot['z'][id]
    obs["line_names"] = wavelength
    return obs


In [28]:
obs = build_obs()

GALAXIE: J003601+003307, ID: 1, PHOTOMETRY: GP_Aperture_Matched_Photometry_v0.fits, EMISSION_LINES: lzlcs_optlines_obs.csv


# Model for input output test, no free variables
what i have:
- redshift
- age
- metallicity
- f_esc

In [29]:
thet = Table.read(os.path.join(path_data, 'galaxies_nostring.csv'), format="ascii") 
thet.add_column([i for i in range(1,67)], name='id', index=0)
thet['f_{esc}'][id]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [30]:
thet['AGE [Myr]']/1000

0.00361
0.00117
0.0039700000000000004
0.00379
0.00632
0.00319
0.006849999999999999
0.0052699999999999995
0.00639
0.0071200000000000005
0.00735


In [31]:
thet

id,OBJID,TYPE,REDSHIFT,AGE [Myr],AGE_ERR,Z[Z_{odot}],Z_ERR,f_{esc},f_ERR_UPPER,f_ERR_LOWER,M_INT_1500[AB],M_INT_1500_ERR,M_OBS_1500[AB],M_OBS_1500_ERR
int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1,3307,0,0.3479,3.61,1.4,0.14,0.08,0.029,nan,nan,-18.64,0.37,-18.53,0.1
2,15440,1,0.3535,1.17,1.12,0.24,0.09,0.013,0.021,-0.003,-21.9,0.45,-20.73,0.1
3,223,1,0.3062,3.97,1.3,0.5,0.19,0.022,0.016,-0.012,-21.46,0.54,-20.27,0.12
4,52044,1,0.3656,3.79,1.73,0.05,0.05,0.038,0.046,-0.016,-20.79,0.44,-20.14,0.1
5,145935,0,0.28,6.32,1.26,0.2,0.12,0.007,nan,nan,-21.82,0.4,-20.44,0.09
6,414608,0,0.2969,3.19,1.03,0.22,0.1,0.004,nan,nan,-21.51,0.37,-19.91,0.08
7,472607,2,0.3565,6.85,1.93,0.1,0.11,0.584,0.416,-0.37,-19.0,0.64,-18.46,0.15
8,414146,1,0.3331,5.27,0.85,0.08,0.05,0.02,0.012,-0.005,-21.11,0.23,-19.52,0.05
9,211459,0,0.2272,6.39,0.75,0.23,0.06,0.007,nan,nan,-23.19,0.17,-20.93,0.04


In [32]:

def build_model(objid=1, thet=thet, fit_el=True, el_table="lzlcs_optlines_obs.csv", phot_table='GP_Aperture_Matched_Photometry_v0.fits', sfh_template="continuity_sfh", add_IGM_model=False, add_duste=False, add_agn=False, add_neb=False,
                nbins_sfh=6, student_t_width=0.3, add_eline_scaling=False, **extras):
    """
    Construct a model.
    sfh_template : "continuity_sfh", "dirichlet_sfh", "parametric_sfh"

    """
    from prospect.models.templates import TemplateLibrary, adjust_continuity_agebins, adjust_dirichlet_agebins
    from prospect.models import priors
    from prospect.models import transforms

    # read in data table

    obs = build_obs(objid=objid, el_table=el_table, phot_table=phot_table, **extras)
    id = obs['cat_row']


    # get SFH template

    if (sfh_template == "continuity_sfh"):
        model_params = TemplateLibrary["continuity_sfh"]
    elif (sfh_template == "parametric_sfh"):
        model_params = TemplateLibrary["parametric_sfh"]


    # IMF: 0: Salpeter (1955); 1: Chabrier (2003); 2: Kroupa (2001)

    model_params['imf_type']['init'] = 1


    # fix redshift

    model_params["zred"]["init"] = obs['z_spec']
    model_params["zred"]["is_free"] = False
    
    # fix metalicity:

    model_params["logzsol"]["init"] = thet['Z[Z_{odot}]'][id]
    model_params["logzsol"]["is_free"] = False

    # fix escape fraction:

    model_params["frac_obrun"] = {"N": 1,
                                      "isfree": False,
                                      "init":thet['f_{esc}'][id], "units": "Fraction of H-ionizing photons that escapes or is dust absorbed."}

    # fix age:

    model_params['tage'] = {"N": 1, "isfree": False,
                                    "init": thet['AGE [Myr]'][id]/1000, "units": "Gyr"}
    

    #model_params["frac_obrun"]["init"] = 
    #model_params["frac_obrun"]["is_free"] = False


    # some functions 

    def zred_to_agebins(zred=model_params["zred"]["init"], agebins=None, z_limit_sfh=10.0, nbins_sfh=6, **extras):
        tuniv = cosmo.age(zred).value*1e9
        tbinmax = tuniv-cosmo.age(z_limit_sfh).value*1e9
        agelims = np.append(np.array([0.0, 6.7, 7.0]), np.linspace(7.0, np.log10(tbinmax), int(nbins_sfh-1))[1:])
        agebins = np.array([agelims[:-1], agelims[1:]])
        return agebins.T
    
    def logmass_to_masses(logmass=None, logsfr_ratios=None, zred=model_params["zred"]["init"], nbins_sfh=6, z_limit_sfh=None, **extras):
        agebins = zred_to_agebins(zred=zred, nbins_sfh=nbins_sfh)
        logsfr_ratios = np.clip(logsfr_ratios, -10, 10)  # numerical issues...
        nbins = agebins.shape[0]
        sratios = 10**logsfr_ratios
        dt = (10**agebins[:, 1]-10**agebins[:, 0])
        coeffs = np.array([(1./np.prod(sratios[:i])) * (np.prod(dt[1:i+1]) / np.prod(dt[:i])) for i in range(nbins)])
        m1 = (10**logmass) / coeffs.sum()
        return m1 * coeffs

    # in the case of continuity, allow scaling with redshift (others fixed redshift)
    print('SFH template =', sfh_template)
    print('redshift init =', model_params["zred"]["init"])

    if (sfh_template == "continuity_sfh"):
        # adjust number of bins for SFH and prior
        model_params['mass']['N'] = nbins_sfh
        model_params['agebins']['N'] = nbins_sfh
        model_params['logsfr_ratios']['N'] = nbins_sfh-1
        model_params['logsfr_ratios']['init'] = np.full(nbins_sfh-1, 0.0)  # constant SFH
        model_params['logsfr_ratios']['prior'] = priors.StudentT(mean=np.full(nbins_sfh-1, 0.0), scale=np.full(nbins_sfh-1, student_t_width), df=np.full(nbins_sfh-1, 2))
        model_params['agebins']['depends_on'] = zred_to_agebins
        # set mass prior
        model_params["logmass"]["prior"] = priors.TopHat(mini=6, maxi=12)
        model_params['mass']['depends_on'] = logmass_to_masses

    elif (sfh_template == "parametric_sfh"):
        model_params["tau"]["prior"] = priors.LogUniform(mini=1e-1, maxi=10)
        model_params["tage"]["prior"] = priors.TopHat(mini=1e-3, maxi=cosmo.age(model_params["zred"]["init"]).value)
        model_params["mass"]["prior"] = priors.LogUniform(mini=1e6, maxi=1e12)


    #complexify the dust
    model_params['dust_type']['init'] = 4
    model_params["dust2"]["prior"] = priors.ClippedNormal(mini=0.0, maxi=4.0, mean=0.3, sigma=1)
    model_params["dust_index"] = {"N": 1,
                                  "isfree": True,
                                  "init": 0.0, "units": "power-law multiplication of Calzetti",
                                  "prior": priors.TopHat(mini=-1.0, maxi=0.4)}

    def to_dust1(dust1_fraction=None, dust1=None, dust2=None, **extras):
        return(dust1_fraction*dust2)

    model_params['dust1'] = {"N": 1,
                             "isfree": False,
                             'depends_on': to_dust1,
                             "init": 0.0, "units": "optical depth towards young stars",
                             "prior": None}
    model_params['dust1_fraction'] = {'N': 1,
                                      'isfree': True,
                                      'init': 1.0,
                                      'prior': priors.ClippedNormal(mini=0.0, maxi=2.0, mean=1.0, sigma=0.3)}

    # fit for IGM absorption
    if add_IGM_model:
        model_params.update(TemplateLibrary["igm"])
        model_params["igm_factor"]['isfree'] = True
        model_params["igm_factor"]["prior"] = priors.ClippedNormal(mean=1.0, sigma=0.3, mini=0.0, maxi=2.0)
    else:
        model_params.update(TemplateLibrary["igm"])
        model_params["igm_factor"]['isfree'] = False

    # Change the model parameter specifications based on some keyword arguments
    if add_duste:
        # Add dust emission (with fixed dust SED parameters)
        model_params.update(TemplateLibrary["dust_emission"])
        model_params['duste_gamma']['isfree'] = True
        model_params['duste_qpah']['isfree'] = True
        model_params['duste_umin']['isfree'] = True

    if add_agn:
        # Add dust emission (with fixed dust SED parameters)
        model_params.update(TemplateLibrary["agn"])
        model_params['fagn']['isfree'] = True
        model_params['agn_tau']['isfree'] = True

    if add_neb:
        # Add nebular emission (with fixed parameters)
        model_params.update(TemplateLibrary["nebular"])
        model_params['gas_logu']['isfree'] = True
        model_params['gas_logz']['isfree'] = True
        _ = model_params["gas_logz"].pop("depends_on")
        #model_params["gas_logz"]["depends_on"] = transforms.stellar_logzsol
        # model_params['nebemlineinspec'] = {'N': 1,
        #                                    'isfree': False,
        #                                    'init': False}

    if add_eline_scaling:
        # Rescaling of emission lines
        model_params["linespec_scaling"] = {"N": 1,
                                            "isfree": True,
                                            "init": 1.0, "units": "multiplative rescaling factor",
                                            "prior": priors.ClippedNormal(mean=1.0, sigma=0.3, mini=0.0, maxi=2.0)}
    
    # Now instantiate the model using this new dictionary of parameter specifications
    if fit_el:
        print("FITTING EL MODEL")
        model = LineSpecModel(model_params)
    else:
        model = SpecModel(model_params)

    return model



In [33]:
model = build_model()
model

GALAXIE: J003601+003307, ID: 1, PHOTOMETRY: GP_Aperture_Matched_Photometry_v0.fits, EMISSION_LINES: lzlcs_optlines_obs.csv
SFH template = continuity_sfh
redshift init = 0.3479409
FITTING EL MODEL


:::::::
<class 'prospect.models.sedmodel.LineSpecModel'>

Free Parameters: (name: prior) 
-----------
  logzsol: <class 'prospect.models.priors.TopHat'>(mini=-2,maxi=0.19)
  dust2: <class 'prospect.models.priors.ClippedNormal'>(mean=0.3,sigma=1,mini=0.0,maxi=4.0)
  logmass: <class 'prospect.models.priors.TopHat'>(mini=6,maxi=12)
  logsfr_ratios: <class 'prospect.models.priors.StudentT'>(mean=[0. 0. 0. 0. 0.],scale=[0.3 0.3 0.3 0.3 0.3],df=[2 2 2 2 2])
  dust_index: <class 'prospect.models.priors.TopHat'>(mini=-1.0,maxi=0.4)
  dust1_fraction: <class 'prospect.models.priors.ClippedNormal'>(mean=1.0,sigma=0.3,mini=0.0,maxi=2.0)

Fixed Parameters: (name: value [, depends_on]) 
-----------
  zred: [0.3479409] 
  mass: [1000000.] <function build_model.<locals>.logmass_to_masses at 0x7f7dc3ad4dc0>
  sfh: [3] 
  imf_type: [1] 
  dust_type: [4] 
  agebins: [[ 0.  8.]
 [ 8.  9.]
 [ 9. 10.]] <function build_model.<locals>.zred_to_agebins at 0x7f7dc3ad6560>
  frac_obrun: [0.029] 
  tage: [0.00361]

In [34]:
x = model.theta_labels
x

<bound method ProspectorParams.theta_labels of :::::::
<class 'prospect.models.sedmodel.LineSpecModel'>

Free Parameters: (name: prior) 
-----------
  logzsol: <class 'prospect.models.priors.TopHat'>(mini=-2,maxi=0.19)
  dust2: <class 'prospect.models.priors.ClippedNormal'>(mean=0.3,sigma=1,mini=0.0,maxi=4.0)
  logmass: <class 'prospect.models.priors.TopHat'>(mini=6,maxi=12)
  logsfr_ratios: <class 'prospect.models.priors.StudentT'>(mean=[0. 0. 0. 0. 0.],scale=[0.3 0.3 0.3 0.3 0.3],df=[2 2 2 2 2])
  dust_index: <class 'prospect.models.priors.TopHat'>(mini=-1.0,maxi=0.4)
  dust1_fraction: <class 'prospect.models.priors.ClippedNormal'>(mean=1.0,sigma=0.3,mini=0.0,maxi=2.0)

Fixed Parameters: (name: value [, depends_on]) 
-----------
  zred: [0.3479409] 
  mass: [1000000.] <function build_model.<locals>.logmass_to_masses at 0x7f7dc3ad4dc0>
  sfh: [3] 
  imf_type: [1] 
  dust_type: [4] 
  agebins: [[ 0.  8.]
 [ 8.  9.]
 [ 9. 10.]] <function build_model.<locals>.zred_to_agebins at 0x7f7dc3a

In [35]:
thetas = model.theta.copy()

In [36]:
def build_sps(zcontinuous=1, sfh_template="continuity_sfh", compute_vega_mags=False, **extras):
    if (sfh_template == "continuity_sfh") or (sfh_template == "dirichlet_sfh"):
        from prospect.sources import FastStepBasis
        sps = FastStepBasis(zcontinuous=zcontinuous,
                            compute_vega_mags=compute_vega_mags,
                            reserved_params=['tage', 'sigma_smooth'])
    else:
        from prospect.sources import CSPSpecBasis
        sps = CSPSpecBasis(zcontinuous=zcontinuous,
                           compute_vega_mags=compute_vega_mags,
                           reserved_params=['sigma_smooth'])
    return sps

In [37]:
for i in range(1,67):
    
    obs=build_obs(objid=i)
    model = build_model(objid=i, fit_el=True)
    thetas = model.theta.copy()

    spec, phot, _ = model.predict(thetas, obs=obs, sps=sps)
    
    phot_unc = np.random.normal(loc=0., scale=np.abs(obs['maggies_unc']))
    spec_unc = np.random.normal(loc=0., scale=np.abs(obs['unc']))
    
    phot_final = phot + phot_unc
    spec_final = spec + spec_unc
    
    phot_dat = Table([obs['filters'], obs['eff_wavelength'], phot_final, np.abs(phot_unc), obs['phot_mask']], 
                      names=['filters', 'eff_wavelength', 'phot', 'phot_unc', 'mask'])
    spec_dat = Table([obs['wavelength'], spec_final, np.abs(spec_unc), obs['mask']], 
                      names=['wavelength', 'spectrum', 'spec_unc', 'mask'])
    
    phot_file =  phot_mock + 'mock_photometry_' + str(i) + '.dat' #'psd_grid_mocks/' + str(ind+1) + '/mock_photometry_' + str(i) + '.dat'
    spec_file =  phot_mock + 'mock_spectrum_' + str(i) + '.dat' #'psd_grid_mocks/' + str(ind+1) + '/mock_spectrum_' + str(i) + '.dat'
    #phot_file = 'mock_data_fixed_highz/mock_photometry_' + str(i) + '.dat'
    #spec_file = 'mock_data_fixed_highz/mock_spectrum_' + str(i) + '.dat'
    
    phot_dat.write(phot_file, format='ascii', overwrite=True)
    spec_dat.write(spec_file, format='ascii', overwrite=True)

GALAXIE: J003601+003307, ID: 1, PHOTOMETRY: GP_Aperture_Matched_Photometry_v0.fits, EMISSION_LINES: lzlcs_optlines_obs.csv
GALAXIE: J003601+003307, ID: 1, PHOTOMETRY: GP_Aperture_Matched_Photometry_v0.fits, EMISSION_LINES: lzlcs_optlines_obs.csv
SFH template = continuity_sfh
redshift init = 0.3479409
FITTING EL MODEL


NameError: name 'sps' is not defined

In [38]:
phot

FITS_rec([( 1, 'J003601+003307',   9.0026448, 5.5201371e-01, 0.3479409, 1,  22.14546  ,  1.45764533e-01, 21.8330574, 0.12754144, 21.80631, 0.11788313, 21.68926, 0.05162214, 20.87918, 0.03594949, 21.94525, 0.12327731, 20.36342, 0.10237351, 21.9752, 0.14799092, 21.7414, 0.13553535, 21.84409, 0.1099972 , 21.71517, 0.04948539, 20.91953, 0.03586105, 21.96557, 0.1171703 , 20.37274, 0.1119778 , 22.94215504, 0.24090529, 22.37951962, 0.11706218,  22.05328862,  1.18646099e-01, 20.93871609, 0.06550245, 19.90560833, 3.49407003e-02, 19.73866383, 5.83357964e-02, 19.1679279 , 0.0672808 , 22.60412396, 0.21564519, 22.22767438, 0.09821946,  21.95356549,  1.16516394e-01, 20.84293355, 4.71949295e-02, 19.8010825 , 2.49730782e-02, 19.64581097, 4.21448772e-02, 19.07253255, 0.04849353, 22.32407803, 0.18167913, 22.01042408, 0.08380543,  21.87155384,  1.28528212e-01, 20.75062222, 3.76840401e-02, 19.69793743, 1.97424341e-02, 19.55271954, 3.36275221e-02, 18.97707981, 3.86090711e-02, 22.14149647, 0.16332669, 21.86

In [39]:
objid = 1
obs = build_obs(objid=objid)
model = build_model_w(objid=objid, fit_el=True)
sps=build_sps()
thetas = model.theta.copy()

spec, phot, _ = model.predict(thetas, obs=obs, sps=sps)

phot_unc = np.random.normal(loc=0., scale=np.abs(obs['maggies_unc']))
spec_unc = np.random.normal(loc=0., scale=np.abs(obs['unc']))

phot_final = phot + phot_unc
spec_final = spec + spec_unc

phot_dat = Table([obs['filters'], obs['eff_wavelength'], phot_final, np.abs(phot_unc), obs['phot_mask']], 
                  names=['filters', 'eff_wavelength', 'phot', 'phot_unc', 'mask'])
spec_dat = Table([obs['wavelength'], spec_final, np.abs(spec_unc), obs['mask']], 
                  names=['wavelength', 'spectrum', 'spec_unc', 'mask'])

phot_file =  phot_mock + 'mock_photometry_' + str(i) + '.dat' #'psd_grid_mocks/' + str(ind+1) + '/mock_photometry_' + str(i) + '.dat'
spec_file =  phot_mock + 'mock_spectrum_' + str(i) + '.dat' #'psd_grid_mocks/' + str(ind+1) + '/mock_spectrum_' + str(i) + '.dat'
#phot_file = 'mock_data_fixed_highz/mock_photometry_' + str(i) + '.dat'
#spec_file = 'mock_data_fixed_highz/mock_spectrum_' + str(i) + '.dat'

phot_dat.write(phot_file, format='ascii', overwrite=True)
spec_dat.write(spec_file, format='ascii', overwrite=True)

GALAXIE: J003601+003307, ID: 1, PHOTOMETRY: GP_Aperture_Matched_Photometry_v0.fits, EMISSION_LINES: lzlcs_optlines_obs.csv


NameError: name 'build_model_w' is not defined

In [40]:
spec

NameError: name 'spec' is not defined

In [41]:
obs['spectrum']

array([ 30.848469 ,  39.194013 ,  61.752844 ,  23.902108 ,  21.293996 ,
        17.924301 ,  32.573245 ,  61.689684 ,  16.382929 ,   0.       ,
       136.51584  , 270.69147  , 851.03992  ,  19.74732  ,   0.       ,
        17.411655 , 454.49678  ,  18.705467 ,   0.       ,   4.8370173])

In [42]:
len(obs['unc'])

20

In [43]:
import time, sys, os
import numpy as np
from astropy.table import Table
from sedpy.observate import load_filters
from prospect import prospect_args
from prospect.fitting import fit_model, lnprobfn
from prospect.io import write_results as writer
from prospect.models.sedmodel import PolySpecModel
#from prospect.models.templates import TemplateLibrary, adjust_continuity_agebins, adjust_dirichlet_agebins, adjust_stochastic_params
from prospect.models import priors, sedmodel, transforms
from astropy.cosmology import Planck15 as cosmo
from prospect.likelihood import NoiseModel
from prospect.likelihood.kernels import Uncorrelated

In [44]:


def build_model_w(objid=0, non_param_sfh=True, dirichlet_sfh=False, stochastic_sfh=False, add_duste=False, add_neb=False, 
                marginalize_neb=False, add_agn=False, n_bins_sfh=10, fit_continuum=False, switch_off_phot=False, 
                switch_off_spec=False, fixed_dust=False, outlier_model=False, **extras):
    
    obs = build_obs(objid=objid)
    
    if non_param_sfh and not dirichlet_sfh and not stochastic_sfh:
        model_params = TemplateLibrary["continuity_sfh"]
    elif dirichlet_sfh:
        model_params = TemplateLibrary["dirichlet_sfh"]
    elif stochastic_sfh:
        model_params = TemplateLibrary["stochastic_sfh"]
    else:
        model_params = TemplateLibrary["parametric_sfh"]
        
    model_params["zred"]['isfree'] = True
    model_params["zred"]["init"] = obs['z_spec']
    model_params["zred"]["prior"] = priors.TopHat(mini=obs['z_spec']-0.005, maxi=obs['z_spec']+0.005)
    
    if non_param_sfh:
        t_univ = cosmo.age(obs['z_spec']).value
        tbinmax = 0.95 * t_univ * 1e9
        #lim1, lim2, lim3, lim4 = 7.4772, 8.0, 8.5, 9.0
        # Early time bins: 1-5 Myr, 5-10, 10-30, 30-100, log...
        lim1, lim2, lim3, lim4 = 6.69897, 7.0, 7.47712, 8.0
        agelims = np.concatenate(([6., lim1, lim2], 
                                  np.log10(np.logspace(lim3, np.log10(tbinmax), n_bins_sfh-3)).flatten().tolist(), 
                                  [np.log10(t_univ*1e9)]))
        if dirichlet_sfh:
            model_params = adjust_dirichlet_agebins(model_params, agelims=agelims)
            model_params["total_mass"]["prior"] = priors.LogUniform(mini=3e9, maxi=1e12)
        
        elif stochastic_sfh:
            agebins = np.array([agelims[:-1], agelims[1:]])
            model_params['agebins']['init'] = agebins.T
            model_params["logmass"]["prior"] = priors.TopHat(mini=9.5, maxi=12.0)
            
            model_params['sigma_reg']['init'] = 0.4
            model_params['sigma_reg']['prior'] = priors.LogUniform(mini=0.1, maxi=5.0)

            model_params['tau_eq']['init'] = 2500/1e3 
            model_params['tau_eq']['prior'] = priors.TopHat(mini=0.01, maxi=t_univ)
            
            model_params['tau_in']['isfree'] = False
            model_params['tau_in']['init'] = t_univ

            model_params['sigma_dyn']['init'] = 0.03 
            model_params['sigma_dyn']['prior'] = priors.LogUniform(mini=0.001, maxi=0.1)
            
            model_params['tau_dyn']['init'] = 25/1e3
            model_params['tau_dyn']['prior'] = priors.ClippedNormal(mini=0.005, maxi=0.2, mean=0.01, sigma=0.02)
            
            model_params = adjust_stochastic_params(model_params, tuniv=t_univ)
            
        else:
            model_params = adjust_continuity_agebins(model_params, tuniv=t_univ, nbins=n_bins_sfh)
            agebins = np.array([agelims[:-1], agelims[1:]])
            model_params['agebins']['init'] = agebins.T
            model_params["logmass"]["prior"] = priors.TopHat(mini=9.5, maxi=12.0)
            
    else:
        model_params["tau"]["prior"] = priors.LogUniform(mini=1e-1, maxi=10)
        model_params["tage"]["prior"] = priors.TopHat(mini=1e-3, maxi=cosmo.age(obs['redshift']).value)
        model_params["mass"]["prior"] = priors.LogUniform(mini=3e9, maxi=1e12)
        model_params["mass"]["init_disp"] = 1e11
    
    if fixed_dust:
        model_params["logzsol"]["prior"] = priors.ClippedNormal(mini=-1.0, maxi=0.19, mean=0.0, sigma=0.15)
    else:
        model_params["logzsol"]["prior"] = priors.TopHat(mini=-1.0, maxi=0.19)
        
    if fixed_dust:
        model_params['dust_type']['init'] = 2
        model_params["dust2"]["prior"] = priors.ClippedNormal(mini=0.0, maxi=4.0, mean=0.3, sigma=1)
        model_params['dust1'] = {"N": 1,
                                "isfree": False,
                                "init": 0.0, "units": "optical depth towards young stars",
                                "prior": None}
    else:
        model_params['dust_type']['init'] = 4
        model_params["dust2"]["prior"] = priors.ClippedNormal(mini=0.0, maxi=4.0, mean=0.3, sigma=1)
        model_params["dust_index"] = {"N": 1,
                                      "isfree": True,
                                      "init": 0.0, "units": "power-law multiplication of Calzetti",
                                      "prior": priors.TopHat(mini=-1.0, maxi=1.0)}
        def to_dust1(dust1_fraction=None, dust1=None, dust2=None, **extras):
            return(dust1_fraction*dust2)
        model_params['dust1'] = {"N": 1,
                                "isfree": False,
                                'depends_on': to_dust1,
                                "init": 0.0, "units": "optical depth towards young stars",
                                "prior": None}
        model_params['dust1_fraction'] = {'N': 1,
                                          'isfree': True,
                                          'init': 1.0,
                                          'prior': priors.ClippedNormal(mini=0.0, maxi=2.0, mean=1.0, sigma=0.3)}
    
    # velocity dispersion
    model_params.update(TemplateLibrary['spectral_smoothing'])
    model_params["sigma_smooth"]["prior"] = priors.TopHat(mini=40.0, maxi=400.0)

    
    if add_duste:
        # Add dust emission (with fixed dust SED parameters)
        model_params.update(TemplateLibrary["dust_emission"])
        model_params['duste_gamma']['isfree'] = True
        model_params['duste_gamma']['init'] = 0.01
        model_params['duste_gamma']['prior'] = priors.LogUniform(mini=1e-4, maxi=0.1)
        model_params['duste_qpah']['isfree'] = True
        model_params['duste_qpah']['prior'] = priors.TopHat(mini=0.5, maxi=7.0)
        model_params['duste_umin']['isfree'] = True
        model_params['duste_umin']['init'] = 1.0
        model_params['duste_umin']['prior'] = priors.ClippedNormal(mini=0.1, maxi=15.0, mean=2.0, sigma=1.0)

    if add_agn:
        # Allow for the presence of an AGN in the mid-infrared
        model_params.update(TemplateLibrary["agn"])
        model_params['fagn']['isfree'] = True
        model_params['fagn']['prior'] = priors.LogUniform(mini=1e-5, maxi=3.0)
        model_params['agn_tau']['isfree'] = True
        model_params['agn_tau']['prior'] = priors.LogUniform(mini=5.0, maxi=150.)

    if add_neb:
        # Add nebular emission
        model_params.update(TemplateLibrary["nebular"])
        model_params['gas_logu']['isfree'] = True
        model_params['gas_logu']['init'] = -2.0
        model_params['gas_logz']['isfree'] = True
        _ = model_params["gas_logz"].pop("depends_on")
        
        # Adjust for widths of emission lines, i.e. gas velocity dispersion
        model_params["nebemlineinspec"]["init"] = False
        model_params["eline_sigma"] = {'N': 1, 
                                       'isfree': True, 
                                       'init': 100.0, 'units': 'km/s',
                                       'prior': priors.TopHat(mini=30, maxi=250)}
        if marginalize_neb:
            model_params.update(TemplateLibrary['nebular_marginalization'])
            model_params['eline_prior_width']['init'] = 1.0
            model_params['use_eline_prior']['init'] = True

    # This removes the continuum from the spectroscopy. Highly recommend using when modeling both photometry & spectroscopy
    if fit_continuum:
        # order of polynomial that's fit to spectrum
        #polyorder_estimate = int(np.clip(np.round((np.min([7500*(obs['redshift']+1), 9150.0])-np.max([3525.0*(obs['redshift']+1), 6000.0]))/(obs['redshift']+1)*100), 10, 30))
        model_params['polyorder'] = {'N': 1,
                                     'init': 10,
                                     'isfree': False}
        
    # This is a pixel outlier model. It helps to marginalize over poorly modeled noise, such as residual sky lines or
    # even missing absorption lines.
    if outlier_model:
        model_params['f_outlier_spec'] = {"N": 1,
                                          "isfree": True,
                                          "init": 0.01,
                                          "prior": priors.TopHat(mini=1e-5, maxi=0.5)}
        model_params['nsigma_outlier_spec'] = {"N": 1,
                                               "isfree": False,
                                               "init": 50.0}
        
    model = PolySpecModel(model_params)
    # model = sedmodel.SedModel(model_params)
    
    return model




In [45]:
model = build_model_w(objid=1)

GALAXIE: J003601+003307, ID: 1, PHOTOMETRY: GP_Aperture_Matched_Photometry_v0.fits, EMISSION_LINES: lzlcs_optlines_obs.csv


NameError: name 'TemplateLibrary' is not defined